In [1]:
from langchain_community.document_loaders import PyPDFLoader
pdf_docs = PyPDFLoader('Data/i-485instr.pdf').load()

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

split_docs=RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=150).split_documents(pdf_docs)
print(len(pdf_docs),len(split_docs))

41 122


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')

d:\Git\LC_First\envlc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Git\LC_First\envlc\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(split_docs, embeddings)
# db.save_local('Data/faiss_index')                         # Save
# db = FAISS.load_local("Data/faiss_index", embeddings)     # Load

In [7]:
retriver = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
#retriver.invoke(f'List of Required evicences for I-485')

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")


from langchain_groq import ChatGroq
llm=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)   #"Llama3-8b-8192"

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
Answer the below question using only the provide context.
Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
parser = StrOutputParser()

rag_chain={"context":retriver,"question":RunnablePassthrough()}|prompt|llm #|parser


In [9]:
rag_chain.invoke('List of evicences to submit with I-485').content

"According to the provided text, the evidence you must submit with Form I-485 includes:\n\n* **Documentation of Your Admission or Parole into the United States:** This includes any travel documents you had, whether you showed them to an immigration inspector, any questions asked, and other details about your admission or parole.\n* **Documentation of Your Immigrant Category:** This means providing evidence showing you are eligible for adjustment of status in a specific category. \n    * If you are filing as a beneficiary of an immigrant petition, you generally need a photocopy of Form I-797, Approval Notice for your petition (or the principal applicant's petition if you are a derivative applicant). \n    * If your Form I-485 is related to an employment-based visa petition (Form I-140) and you are filing after the employer filed the Form I-140 on your behalf, you *must* file Form I-485 Supplement J, Confirmation of Bona Fide Job Offer or Request for Job Portability under INA Section 204

In [ ]:
#retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

